In [1]:
from tools.audio_recorder import AudioRecorder
from tools.audio_transcriber import AudioTranscriber
from tools.ai_chatbot import AIChatbot  # Import AIChat class
import keyboard

def listen_for_key():
    recorder = AudioRecorder()
    transcriber = AudioTranscriber()
    chatbot = AIChatbot(api_key_file='api.txt', assistant_name="John")  # Instantiate AIChat

    try:
        print("Press and hold the spacebar to record.")
        print("Release the spacebar to stop recording.")
        print("Press Esc to terminate the program.")

        while True:
            if keyboard.is_pressed('space'):
                if not recorder.recording:
                    recorder.start_recording()

            elif recorder.recording:
                recorder.stop_recording()
                transcribed_text = transcriber.transcribe_audio(recorder.OUTPUT_FILENAME)

                ai_response = chatbot.ask(transcribed_text)
                print("AI Response:", ai_response)

            if keyboard.is_pressed('esc'):
                if recorder.recording:
                    recorder.stop_recording()
                print("Terminating program...")
                break
    finally:
        recorder.cleanup()

if __name__ == "__main__":
    listen_for_key()


c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

Press and hold the spacebar to record.
Release the spacebar to stop recording.
Press Esc to terminate the program.
Recording started...
Recording saved to ./recordings\output.wav
Transcribing ./recordings\output.wav...
Transcription completed.
 Hello, how are you?
AI Response: Hi there! I'm John, your friendly AI assistant. I don't have emotions or feelings like humans, but I'm ready to assist you with any questions or tasks you might have. How can I help you today?
Recording started...
Recording saved to ./recordings\output.wav
Transcribing ./recordings\output.wav...
Transcription completed.
 What is your name?
AI Response: My name is John. How can I help you today?


KeyboardInterrupt: 